In [54]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import string


def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)
    
print "Reading data..."
data = list(parseData("beer_50000.json"))[:5000]
print "done"
punctuation = set(string.punctuation)

Reading data...
done


In [55]:
print 'Question 1:'
bigramCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    wl = r.split()
    if len(wl) > 1:
        for i in range(len(wl)-1):
            bg = wl[i] + ' ' + wl [i+1]
            bigramCount[bg] += 1

mostPop =  [(bigramCount[bg],bg)for bg in bigramCount.keys()]
mostPop.sort()
mostPop.reverse()
print 'Answer 1:'
print mostPop[:5]
print 'numbre of unique bigrams: ', len(bigramCount.keys()) 

Question 1:
Answer 1:
[(4587, 'with a'), (2595, 'in the'), (2245, 'of the'), (2056, 'is a'), (2033, 'on the')]
numbre of unique bigrams:  182246


In [56]:
print 'Question 2:'
bigrs = [mp[1] for mp in mostPop[:1000]]
bigrsId = dict(zip(bigrs, range(len(bigrs))))
bigrsSet = set(bigrs)

def getbgs(wordlist):
  bgs = []  
  for i in range(len(wordlist)-1):
    s = wordlist[i] + ' ' + wordlist[i+1]
    bgs.append(s)
  return bgs

def featureQ2(datum):
  feat = [0]*1000
  r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
  bgs = getbgs(r.split())
  for b in bgs:
    if b in bigrsSet:
      feat[bigrsId[b]] += 1
  feat.append(1) #offset
  return feat

X = [featureQ2(d) for d in data]
y = [d['review/overall'] for d in data]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
predictions = clf.predict(X)
print "MSE: " + str(mean_squared_error(predictions,y))


Question 2:
MSE: 0.343153014061


In [57]:
print 'Question 3:'

def getbgs(wordlist):
  bgs = []  
  for i in range(len(wordlist)-1):
    s = wordlist[i] + ' ' + wordlist[i+1]
    bgs.append(s)
  return bgs

b_n_u = defaultdict(int)
punctuation = set(string.punctuation)

for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  wordlist = r.split()
  for w in wordlist:
    b_n_u[w] += 1
  blist = getbgs(wordlist)
  for s in blist:
    b_n_u[s] += 1

bcounts = [(b_n_u[w], w) for w in b_n_u.keys()]
bcounts.sort()
bcounts.reverse()

mix1000 = [x[1] for x in bcounts[:1000]]
mixgramId = dict(zip(mix1000, range(len(mix1000))))
mix1000Set = set(mix1000)

def featureQ3(datum):
  feat = [0]*1000
  r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
  wordlist = r.split()
  for w in wordlist:
    if w in mix1000Set:
      feat[mixgramId[w]] += 1
  blist = getbgs(wordlist)
  for b in blist:
    if b in mix1000Set:
      feat[mixgramId[b]] += 1
  feat.append(1) #offset
  return feat

X = [featureQ3(d) for d in data]
y = [d['review/overall'] for d in data]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
predictions = clf.predict(X)
theta = clf.coef_
print "MSE " + str(mean_squared_error(predictions,y))

Question 3:
MSE 0.289047333034


In [58]:
print 'Question 4:'
weights = zip (theta[:1000],range(len(theta[:1000])))
weights.sort()
neg = [mix1000[weights[i][1]] for i in range(5)]
print 'Negative: ', neg
weights.reverse()
pos = [mix1000[weights[i][1]] for i in range(5)]
print 'Positive: ', pos

Question 4:
Negative:  ['sort of', 'water', 'corn', 'the background', 'straw']
Positive:  ['sort', 'a bad', 'of these', 'not bad', 'the best']


In [59]:
import math
print 'Question 5:'
idf = defaultdict(float)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in set(r.split()):
    idf[w] += 1
docs = len(data)
for k in idf:
  idf[k] = math.log(docs/float(idf[k]))/math.log(10)
t = ['foam', 'smell', 'banana', 'lactic', 'tart']
tfidf = defaultdict(float)
for w in t:
  print "idf " + w + " : " + str(idf[w])
d = data[0]
tf = defaultdict(float)
r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
for w in r.split():
  if w in t:
    tf[w] += 1
for w in t:
    tf[w] = float(tf[w])/len(r.split())
    tfidf[w] = tf[w]*idf[w]
tfidf = dict(tfidf)
print tfidf

Question 5:
idf foam : 1.13786862069
idf smell : 0.537901618865
idf banana : 1.67778070527
idf lactic : 2.92081875395
idf tart : 1.80687540165
{'foam': 0.046443617170896435, 'smell': 0.010977584058466208, 'banana': 0.06848084511290124, 'lactic': 0.11921709199805611, 'tart': 0.03687500819684771}


In [60]:
print 'Question 6:'

def idf(t):
    docfreq = 0
    for d in data:
        r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
        if t in r.split():
            docfreq += 1
    return  math.log(len(data)/float(docfreq))/math.log(10)

def tf(t, i):
    freq = 0
    r = ''.join([c for c in data[i]['review/text'].lower() if not c in punctuation])
    if len(r.split())> 0:
        for w in r.split():
            if w == t:
                freq += 1
        freq =  float(freq)/len(r.split())
    return freq

uniCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    for w in r.split():
        uniCount[w] += 1
counts = [(uniCount[w], w) for w in uniCount]
counts.sort()
counts.reverse()
uni1000 = [x[1] for x in counts[:1000]]
uni1000_idf = defaultdict(float)
for w in uni1000:
    uni1000_idf[w] = idf(w)
tfidfrv1 = [tf(w, 0) * uni1000_idf[w] for w in uni1000]
tfidfrv2 = [tf(w, 1) * uni1000_idf[w] for w in uni1000]
dot_prod = sum(tfidfrv1[i] * tfidfrv2[i] for i in range(1000))
two_norm_prod = numpy.sqrt(sum(tfidfrv1[i]**2 for i in range(1000)))*numpy.sqrt(sum(tfidfrv2[i]**2 for i in range(1000)))
cs = float(dot_prod) / two_norm_prod
print "cosine similariy: " + str(cs)

Question 6:
cosine similariy: 0.106130241679


In [61]:
print 'Question 7:'
maxi = 0
maxcs = -1000
tidrv1 = [tf(w, 0) * uni1000_idf[w] for w in uni1000]
for i in range(1, len(data)):
    tidrv2 = [tf(w, i) * uni1000_idf[w] for w in uni1000]
    dot_prod = sum(tidrv1[i] * tidrv2[i] for i in range(1000))
    two_norm_prod = numpy.sqrt(sum(tidrv1[i]**2 for i in range(1000))) * numpy.sqrt(sum(tidrv2[i]**2 for i in range(1000)))
    cs = float(dot_prod) /two_norm_prod 
    if cs > maxcs:
        maxcs = cs
        maxi = i
print "beer most similar to first : " 
print 'beerId: ', data[maxi]['beer/beerId']
print 'profileName: ', data[maxi]['user/profileName']



Question 7:


C:\Users\Khor_000\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


beer most similar to first : 
beerId:  52211
profileName:  Heatwave33


In [62]:
print 'Question 8:'
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words1000 = [x[1] for x in counts[:1000]]
wordId = dict(zip(words1000, range(len(words1000))))
idf = defaultdict(float)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in set(r.split()):
    idf[w] += 1
docs = len(data)
for k in idf:
  idf[k] = math.log(docs/float(idf[k]))/math.log(10)


def featureQ8(datum):
  feat = [0]*1000
  r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
  if len(r.split())> 0:
      for w in r.split():
        if w in words1000:
          feat[wordId[w]] += idf[w]
      featd = [  float(f)/len(r.split())  for f in feat] 
      featd.append(1)
      return featd
  feat.append(1) #offset
  return feat

X = [featureQ8(d) for d in data]
y = [d['review/overall'] for d in data]
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
print "MSE: " + str(mean_squared_error(predictions,y))


Question 8:
MSE: 0.508110529164
